<a href="https://colab.research.google.com/github/nurikahraman/FoodTracker/blob/main/COCO_to_YOLO_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#             ♻**COCO to YOLO Conversion**

# **Google Drive, Klasör Konum Ayarlamaları**

In [15]:
!echo "Google Drive deposu baglaniyor..."
from google.colab import drive
drive.mount('/content/drive')

#Datasets (This gets downloaded in "/content/foodChallenge/data")
!echo "🗄 Preparing the validation dataset..."
!unzip -q "/content/drive/MyDrive/Colab Notebooks/content/FoodRecognitionChallenge/val.zip" -d "/content/dataset/val"

!echo "🗄 Preparing the dataset for training..."
!unzip -q "/content/drive/MyDrive/Colab Notebooks/content/FoodRecognitionChallenge/train.zip" -d "/content/dataset/train"

Google Drive deposu baglaniyor...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🗄 Preparing the validation dataset...
replace /content/dataset/val/annotations.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace /content/dataset/val/images/006452.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 🗄 Preparing the dataset for training...
replace /content/dataset/train/annotations.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 
COCO json dosyası konumu ve YOLO dosyası formatı çıktı konum ayarı.



In [33]:
!echo ""
!echo "COCO json dosyası konumu / YOLO formatlı çıktı konumu ayarlandı."
!echo ""

!mkdir -p '/content/dataset/val'
!mkdir -p '/content/dataset/train'
!mkdir -p '/content/dataset/YOLO_Dataset/train/images'
!mkdir -p '/content/dataset/YOLO_Dataset/train/labels'
!mkdir -p '/content/dataset/YOLO_Dataset/val/images'
!mkdir -p '/content/dataset/YOLO_Dataset/val/labels'
!mkdir -p '/content/dataset/YOLO_Dataset/test/images'
!mkdir -p '/content/dataset/YOLO_Dataset/test/labels'
!mkdir -p "/content/drive/MyDrive/Colab Notebooks/Aptal Buzdolabı/outputs/yolo_dataset/"

input_path = "/content/dataset/"
output_path = "/content/dataset/YOLO_Dataset/"
train_images = "/content/dataset/train/images/"
COCO_Dataset = "/content/dataset/train/annotations.json"


COCO json dosyası konumu ve YOLO dosyası formatı çıktı konum ayarı.



# **Convert Routines**
Kaynak:
https://medium.com/red-buffer/converting-a-custom-dataset-from-coco-format-to-yolo-format-6d98a4fd43fc

In [34]:
#@title ÖN KURULUM
import json
import cv2
import os
import matplotlib.pyplot as plt
import shutil

In [35]:
#@title JSON Ek Açıklama dosyasını okuma
!echo "JSON Ek Açıklama dosyası okundu"
f = open(COCO_Dataset)
data = json.load(f)
f.close()

JSON Ek Açıklama dosyası okundu


In [ ]:
#@title Görüntülerin İşlenmesi
!echo "Görüntülerin İşlenmesi"

file_names = []

def load_images_from_folder(folder):
  count = 0
  for filename in os.listdir(folder):
        source = os.path.join(folder,filename)
        destination = f"{output_path}images/img{count}.jpg"

        try:
            shutil.copy(source, destination)
            print(count, "File copied successfully.")
        # If source and destination are same
        except shutil.SameFileError:
            print("Source and destination represents the same file.")

        file_names.append(filename)
        count += 1

load_images_from_folder(train_images)

In [36]:
#@title Yardımcı İşlevler
def get_img_ann(image_id):
    img_ann = []
    isFound = False
    for ann in data['annotations']:
        if ann['image_id'] == image_id:
            img_ann.append(ann)
            isFound = True
    if isFound:
        return img_ann
    else:
        return None

In [37]:
def get_img(filename):
  for img in data['images']:
    if img['file_name'] == filename:
      return img

In [38]:
count = 0

for filename in file_names:
  # Extracting image
  img = get_img(filename)
  img_id = img['id']
  img_w = img['width']
  img_h = img['height']

  # Get Annotations for this image
  img_ann = get_img_ann(img_id)

  if img_ann:
    # Opening file for current image
    file_object = open(f"{output_path}labels/img{count}.txt", "a")

    for ann in img_ann:
      current_category = ann['category_id'] - 1 # As yolo format labels start from 0
      current_bbox = ann['bbox']
      x = current_bbox[0]
      y = current_bbox[1]
      w = current_bbox[2]
      h = current_bbox[3]

      # Finding midpoints
      x_centre = (x + (x+w))/2
      y_centre = (y + (y+h))/2

      # Normalization
      x_centre = x_centre / img_w
      y_centre = y_centre / img_h
      w = w / img_w
      h = h / img_h

      # Limiting upto fix number of decimal places
      x_centre = format(x_centre, '.6f')
      y_centre = format(y_centre, '.6f')
      w = format(w, '.6f')
      h = format(h, '.6f')

      # Writing current object
      file_object.write(f"{current_category} {x_centre} {y_centre} {w} {h}\n")

    file_object.close()
    count += 1  # This should be outside the if img_ann block.

# **Roboflow Upload Routine**

In [ ]:
!pip install roboflow
!roboflow login
!roboflow import -w akilli-atolye -p test-ppapj "/content/drive/MyDrive/Colab Notebooks/Aptal Buzdolabı/outputs/yolo_dataset"